# TP2
### Équipe 10
### Fabrice Charbonneau (1798064)
### Sanou Armel Kenzanga Landry (1976804)


Tout d'abord, lire les fichiers et les ajouter à un dataframe

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_raw = pd.DataFrame(columns=['uni', 'course', 'text'])

In [ ]:
i = 0
for file in list